# MedicAL: Literature Review Model Training✨
------------------

## 1 - Install and Import Necessary Libraries 💥




In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BlenderbotTokenizer, BlenderbotForConditionalGeneration
import torch
from sklearn.model_selection import train_test_split
import os
import pickle
import logging
import torch
from transformers import AutoConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import PreTrainedTokenizer, MODEL_WITH_LM_HEAD_MAPPING, WEIGHTS_NAME
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from pathlib import Path
from typing import Dict, List, Tuple

# Caching and storing of data/checkpoints
import json
import re
import random
import glob
import shutil
import numpy as np

Check if cuda is empty or not.

In [ ]:
torch.cuda.empty_cache()

## 2 - Preprocessing Step 👻

In [ ]:
df = pd.read_json("/content/documents.json")
df = df.drop(df.index[0])
df

,page_content,metadata,type
1,ANEW ENGLAND GUIDE FOR PATIENTS AND HEALTH PRO...,{'question': 'Question: What organization publ...,Document
2,"Beverly C. Burke,MSW , Co-Chair\nLead Planner/...",{'question': 'What organizations and individua...,Document
3,"Ferre Institute\nW. Andr ew Faucett, MS, CGC, ...",{'question': 'Who is the Chief of the Genetic ...,Document
4,Preface 3\nChapter 1 Genetics 101 5\n1.1 ...,{'question': 'What are the major types of gene...,Document
5,"Chapter 8 Ethical, Legal and Social Issues 39\...","{'question': 'What are some of the ethical, le...",Document
...,...,...,...
11316,"240 Index\nSPIRIV A (tiotropium), 52, 256\nspi...",{'question': 'What is the mechanism of action ...,Document
11317,"TAVIST (clemastine), 240TAXOL (paclitaxel), 20...",{'question': 'What is the therapeutic index of...,Document
11318,"tizanidine, 64TNKase (tenecteplase), 101tobram...",{'question': 'What is the indication for the u...,Document
11319,"241 Index\ntriptorelin, 212, 229\ntroleandomy...",{'question': 'What is the index number for tro...,Document


In [ ]:
df['question'] = df['metadata'].apply(lambda x: x['question'])
df['answer'] = df['metadata'].apply(lambda x: x['answer'])

df

,page_content,metadata,type,question,answer
1,ANEW ENGLAND GUIDE FOR PATIENTS AND HEALTH PRO...,{'question': 'Question: What organization publ...,Document,"Question: What organization published the ""A N...",The New England Guide for Patients and Health ...
2,"Beverly C. Burke,MSW , Co-Chair\nLead Planner/...",{'question': 'What organizations and individua...,Document,What organizations and individuals were involv...,This text provides a list of individuals and o...
3,"Ferre Institute\nW. Andr ew Faucett, MS, CGC, ...",{'question': 'Who is the Chief of the Genetic ...,Document,Who is the Chief of the Genetic Services Branc...,The Ferre Institute includes instructors and c...
4,Preface 3\nChapter 1 Genetics 101 5\n1.1 ...,{'question': 'What are the major types of gene...,Document,What are the major types of genetic diseases d...,"This text provides an overview of genetics, di..."
5,"Chapter 8 Ethical, Legal and Social Issues 39\...","{'question': 'What are some of the ethical, le...",Document,"What are some of the ethical, legal, and socia...","The text discusses various ethical, legal, and..."
...,...,...,...,...,...
11316,"240 Index\nSPIRIV A (tiotropium), 52, 256\nspi...",{'question': 'What is the mechanism of action ...,Document,What is the mechanism of action of SPIRIVA (ti...,The index includes various medications such as...
11317,"TAVIST (clemastine), 240TAXOL (paclitaxel), 20...",{'question': 'What is the therapeutic index of...,Document,What is the therapeutic index of TAVIST (clema...,The literature review includes information on ...
11318,"tizanidine, 64TNKase (tenecteplase), 101tobram...",{'question': 'What is the indication for the u...,Document,What is the indication for the use of Toviaz (...,The text provides a list of various medication...
11319,"241 Index\ntriptorelin, 212, 229\ntroleandomy...",{'question': 'What is the index number for tro...,Document,What is the index number for trovaﬂoxacin in t...,The index includes various medications such as...


In [ ]:
df = df.drop(columns=['type'])
df

,page_content,metadata,question,answer
1,ANEW ENGLAND GUIDE FOR PATIENTS AND HEALTH PRO...,{'question': 'Question: What organization publ...,"Question: What organization published the ""A N...",The New England Guide for Patients and Health ...
2,"Beverly C. Burke,MSW , Co-Chair\nLead Planner/...",{'question': 'What organizations and individua...,What organizations and individuals were involv...,This text provides a list of individuals and o...
3,"Ferre Institute\nW. Andr ew Faucett, MS, CGC, ...",{'question': 'Who is the Chief of the Genetic ...,Who is the Chief of the Genetic Services Branc...,The Ferre Institute includes instructors and c...
4,Preface 3\nChapter 1 Genetics 101 5\n1.1 ...,{'question': 'What are the major types of gene...,What are the major types of genetic diseases d...,"This text provides an overview of genetics, di..."
5,"Chapter 8 Ethical, Legal and Social Issues 39\...","{'question': 'What are some of the ethical, le...","What are some of the ethical, legal, and socia...","The text discusses various ethical, legal, and..."
...,...,...,...,...
11316,"240 Index\nSPIRIV A (tiotropium), 52, 256\nspi...",{'question': 'What is the mechanism of action ...,What is the mechanism of action of SPIRIVA (ti...,The index includes various medications such as...
11317,"TAVIST (clemastine), 240TAXOL (paclitaxel), 20...",{'question': 'What is the therapeutic index of...,What is the therapeutic index of TAVIST (clema...,The literature review includes information on ...
11318,"tizanidine, 64TNKase (tenecteplase), 101tobram...",{'question': 'What is the indication for the u...,What is the indication for the use of Toviaz (...,The text provides a list of various medication...
11319,"241 Index\ntriptorelin, 212, 229\ntroleandomy...",{'question': 'What is the index number for tro...,What is the index number for trovaﬂoxacin in t...,The index includes various medications such as...


In [ ]:
df.dtypes

page_content    object
metadata        object
question        object
answer          object
dtype: object

In [ ]:
df = df.astype(str)

In [ ]:
df

,page_content,metadata,question,answer
1,ANEW ENGLAND GUIDE FOR PATIENTS AND HEALTH PRO...,{'question': 'Question: What organization publ...,"Question: What organization published the ""A N...",The New England Guide for Patients and Health ...
2,"Beverly C. Burke,MSW , Co-Chair\nLead Planner/...",{'question': 'What organizations and individua...,What organizations and individuals were involv...,This text provides a list of individuals and o...
3,"Ferre Institute\nW. Andr ew Faucett, MS, CGC, ...",{'question': 'Who is the Chief of the Genetic ...,Who is the Chief of the Genetic Services Branc...,The Ferre Institute includes instructors and c...
4,Preface 3\nChapter 1 Genetics 101 5\n1.1 ...,{'question': 'What are the major types of gene...,What are the major types of genetic diseases d...,"This text provides an overview of genetics, di..."
5,"Chapter 8 Ethical, Legal and Social Issues 39\...","{'question': 'What are some of the ethical, le...","What are some of the ethical, legal, and socia...","The text discusses various ethical, legal, and..."
...,...,...,...,...
11316,"240 Index\nSPIRIV A (tiotropium), 52, 256\nspi...",{'question': 'What is the mechanism of action ...,What is the mechanism of action of SPIRIVA (ti...,The index includes various medications such as...
11317,"TAVIST (clemastine), 240TAXOL (paclitaxel), 20...",{'question': 'What is the therapeutic index of...,What is the therapeutic index of TAVIST (clema...,The literature review includes information on ...
11318,"tizanidine, 64TNKase (tenecteplase), 101tobram...",{'question': 'What is the indication for the u...,What is the indication for the use of Toviaz (...,The text provides a list of various medication...
11319,"241 Index\ntriptorelin, 212, 229\ntroleandomy...",{'question': 'What is the index number for tro...,What is the index number for trovaﬂoxacin in t...,The index includes various medications such as...


In [ ]:
df.dtypes

page_content    object
metadata        object
question        object
answer          object
dtype: object

## 3 - Split Dataset into Train-Validation 📈

In [ ]:
trn_df, val_df = train_test_split(df, test_size=0.2, random_state=42, shuffle=True)
# Would like to sample your data to prevent memory error
# trn_df = trn_df.sample(1500)
# val_df = val_df.sample(300)

logger = logging.getLogger(__name__)

def construct_convo(row, tokenizer):
    conv = list(reversed([tokenizer.encode(x) + [tokenizer.eos_token_id] for x in row]))
    flatten = lambda l: [item for sublist in l for item in sublist]
    conv = flatten(conv)
    return conv

We will use "microsoft/DialoGPT-small" as our tokenizer.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
for _, row in trn_df.iterrows():
  conv = construct_convo(row, tokenizer)
  break

## 4 - Training our Model 🤗

Create our dataset function.

In [ ]:
class ConversationDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, df, block_size=512):
        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)

        directory = args.cache_dir
        cached_features_file = os.path.join(
            directory, args.model_type + "_cached_lm_" + str(block_size)
        )

        if os.path.exists(cached_features_file) and not args.overwrite_cache:
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)

            self.examples = []
            for _, row in df.iterrows():
                conv = construct_convo(row, tokenizer)
                self.examples.append(conv)

            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

In [ ]:
def load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False):
   return ConversationDataset(tokenizer, args, df_val if evaluate else df_trn)

In [ ]:
def set_seed(args):
   random.seed(args.seed)
   np.random.seed(args.seed)
   torch.manual_seed(args.seed)
   if args.n_gpu > 0:
       torch.cuda.manual_seed_all(args.seed)

In [ ]:
def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
   ordering_and_checkpoint_path = []


   glob_checkpoints = glob.glob(os.path.join(args.output_dir, "{}-*".format(checkpoint_prefix)))


   for path in glob_checkpoints:
       if use_mtime:
           ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
       else:
           regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
           if regex_match and regex_match.groups():
               ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))


   checkpoints_sorted = sorted(ordering_and_checkpoint_path)
   checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
   return checkpoints_sorted

In [ ]:
def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
   if not args.save_total_limit:
       return
   if args.save_total_limit <= 0:
       return


   # Check if we should delete older checkpoint(s)
   checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
   if len(checkpoints_sorted) <= args.save_total_limit:
       return

   number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args.save_total_limit)
   checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
   for checkpoint in checkpoints_to_be_deleted:
       logger.info("Deleting older checkpoint [{}] due to args.save_total_limit".format(checkpoint))
       shutil.rmtree(checkpoint)

from transformers import PreTrainedModel, AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm, trange

try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

Create our train function.

In [ ]:
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    if args.local_rank in [-1, 0]:
        tb_writer = SummaryWriter()

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args.train_batch_size, collate_fn=collate, drop_last = True
    )
    print("Batch size: ", args.train_batch_size)
    print("Data Loader length: ", len(train_dataloader))

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    model = model.module if hasattr(model, "module") else model  # Take care of distributed/parallel training
    model.resize_token_embeddings(len(tokenizer))


    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args.weight_decay,
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
    )

    # Check if saved optimizer or scheduler states exist
    if (
        args.model_name_or_path
        and os.path.isfile(os.path.join(args.model_name_or_path, "optimizer.pt"))
        and os.path.isfile(os.path.join(args.model_name_or_path, "scheduler.pt"))
    ):
        # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args.model_name_or_path, "scheduler.pt")))

    if args.fp16:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)

    # multi-gpu training (should be after apex fp16 initialization)
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Distributed training (should be after apex fp16 initialization)
    if args.local_rank != -1:
        model = torch.nn.parallel.DistributedDataParallel(
            model, device_ids=[args.local_rank], output_device=args.local_rank, find_unused_parameters=True
        )

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Instantaneous batch size per GPU = %d", args.per_gpu_train_batch_size)
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args.train_batch_size
        * args.gradient_accumulation_steps
        * (torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args.model_name_or_path and os.path.exists(args.model_name_or_path):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args.model_name_or_path.split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args.gradient_accumulation_steps)
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args.gradient_accumulation_steps)

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(
        epochs_trained, int(args.num_train_epochs), desc="Epoch", disable=args.local_rank not in [-1, 0]
    )
    set_seed(args)  # Added here for reproducibility
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=args.local_rank not in [-1, 0])
        for step, batch in enumerate(epoch_iterator):
            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch)
            if inputs.shape[1] > 1024: continue
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            model.train()
            outputs = model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args.n_gpu > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()
            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args.local_rank in [-1, 0] and args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    if (
                        args.local_rank == -1 and args.evaluate_during_training
                    ):  # Only evaluate when single GPU otherwise metrics may not average well
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar("eval_{}".format(key), value, global_step)
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args.logging_steps, global_step)
                    logging_loss = tr_loss

                if args.local_rank in [-1, 0] and args.save_steps > 0 and global_step % args.save_steps == 0:
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args.output_dir, "{}-{}".format(checkpoint_prefix, global_step))
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = (
                        model.module if hasattr(model, "module") else model
                    )  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix)

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if args.max_steps > 0 and global_step > args.max_steps:
                print("Batch shape: ", batch.shape)
                epoch_iterator.close()
                break
        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    if args.local_rank in [-1, 0]:
        tb_writer.close()

    return global_step, tr_loss / global_step

Let's create our evaluation function.

In [ ]:
# Evaluation of some model

def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, df_trn, df_val, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True)
    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size, collate_fn=collate, drop_last = True
    )

    # multi-gpu evaluate
    if args.n_gpu > 1:
        model = torch.nn.DataParallel(model)

    # Evaluation
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        inputs, labels = (batch, batch)
        inputs = inputs.to(args.device)
        labels = labels.to(args.device)

        with torch.no_grad():
            outputs = model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result


Lastly, initalize our arguments.

In [ ]:
class Args():
    def __init__(self):
        self.output_dir = 'medical-literature-small'
        self.model_type = 'gpt2'
        self.model_name_or_path = 'microsoft/DialoGPT-small'
        self.config_name = 'microsoft/DialoGPT-small'
        self.tokenizer_name = 'microsoft/DialoGPT-small'
        self.cache_dir = 'cached'
        self.block_size = 512
        self.do_train = True
        self.do_eval = True
        self.evaluate_during_training = False
        self.per_gpu_train_batch_size = 2
        self.per_gpu_eval_batch_size = 2
        self.gradient_accumulation_steps = 1
        self.learning_rate = 5e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 3
        self.max_steps = -1
        self.warmup_steps = 0
        self.logging_steps = 1000
        self.save_steps = 3500
        self.save_total_limit = None
        self.eval_all_checkpoints = False
        self.no_cuda = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.should_continue = False
        self.seed = 42
        self.local_rank = -1
        self.fp16 = False
        self.fp16_opt_level = 'O1'


args = Args()

Now, we need to combine all of our methods to get ready for training! 👩🏽‍💻

In [ ]:
# Main runner
def main(df_trn, df_val):
    args = Args()

    if args.should_continue:
        sorted_checkpoints = _sorted_checkpoints(args)
        if len(sorted_checkpoints) == 0:
            raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
        else:
            args.model_name_or_path = sorted_checkpoints[-1]

    if (
            os.path.exists(args.output_dir)
            and os.listdir(args.output_dir)
            and args.do_train
            and not args.overwrite_output_dir
            and not args.should_continue
    ):
        raise ValueError(
            "Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(
                args.output_dir
            )
        )

    # Setup CUDA, GPU & distributed training
    device = torch.device("cuda")
    args.n_gpu = torch.cuda.device_count()
    args.device = device

    # Setup logging
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN,
    )
    logger.warning(
        "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
        args.local_rank,
        device,
        args.n_gpu,
        bool(args.local_rank != -1),
        args.fp16,
    )

    # Set seed
    set_seed(args)

    config = AutoConfig.from_pretrained(args.config_name, cache_dir=args.cache_dir)
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, cache_dir=args.cache_dir, max_length=10000)
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=False,
        config=config,
        cache_dir=args.cache_dir,
    )
    model.to(args.device)

    logger.info("Training/evaluation parameters %s", args)

    # Training
    if args.do_train:
        train_dataset = load_and_cache_examples(args, tokenizer, df_trn, df_val, evaluate=False)
        print(train_dataset)
        print("Train Dataset Length: ", len(train_dataset))
        global_step, tr_loss = train(args, train_dataset, model, tokenizer)
        logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

    if args.do_train:
        # Create output directory if needed
        os.makedirs(args.output_dir, exist_ok=True)

        logger.info("Saving model checkpoint to %s", args.output_dir)

        model_to_save = (
            model.module if hasattr(model, "module") else model
        )  # Take care of distributed/parallel training
        model_to_save.save_pretrained(args.output_dir)
        tokenizer.save_pretrained(args.output_dir)

        torch.save(args, os.path.join(args.output_dir, "training_args.bin"))

        # Load a trained model and vocabulary that you have fine-tuned
        model = AutoModelForCausalLM.from_pretrained(args.output_dir)
        tokenizer = AutoTokenizer.from_pretrained(args.output_dir, max_length=10000)
        model.to(args.device)

    # Evaluation
    results = {}
    if args.do_eval and args.local_rank in [-1, 0]:
        checkpoints = [args.output_dir]
        if args.eval_all_checkpoints:
            checkpoints = list(
                os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + "/**/" + WEIGHTS_NAME, recursive=True))
            )
            logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
        logger.info("Evaluate the following checkpoints: %s", checkpoints)
        for checkpoint in checkpoints:
            global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
            prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""

            model = AutoModelForCausalLM.from_pretrained(checkpoint)
            model.to(args.device)
            result = evaluate(args, model, tokenizer, df_trn, df_val, prefix=prefix)
            result = dict((k + "_{}".format(global_step), v) for k, v in result.items())
            results.update(result)

    return results

And... Here we go!💫

In [ ]:
main(trn_df, val_df)

config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 213.06 MiB is free. Process 9324 has 14.54 GiB memory in use. Of the allocated memory 14.43 GiB is allocated by PyTorch, and 13.52 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## 5 - Try Your Model 🎯

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelForCausalLM.from_pretrained('medical-literature-small')

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(tokenizer.eos_token + input(">> User:"), return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens,
    chat_history_ids = model.generate(
        bot_input_ids,
        max_length=1000,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=3,
        do_sample=True,
        top_k=100,
        top_p=0.7,
        temperature = 0.1,
                                      )

    # pretty print last ouput tokens from bot
    print("Medical AI: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0]).replace("<|endoftext|>", "")))

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

>> User:what is hematology
Medical AI:  and what is the significance of hematologic diagnosis in the treatment of hepatologic conditions?
>> User:answer to this question: what is cardiology
Medical AI:  and what are the significance and significance of his diagnosis in treatment ofhepatologic condition?
Answer to the question: What is hematic diagnosis and what does hematological diagnosis in this treatment of hemolytic anemia?
Question to the answer: What are the cardinal directions of hematic disease?
answer to the questions: What hematologists and what do they represent?
Answers to the answers to the previous questions:
Answer: What you need to know is hemediology and the significance
of hematologic diagnosis in
the treatment of Hemolytic Anemias.
An answer to the first question is:
What is hemologic diagnosis and why is it important?
The answer to this answer is: What
is hematomy and what
is the significance or significance of the hematologist diagnosis in hematolytic
anemia?
>> Us

## 6 - Save Your Model To Hugging Face Hub! ⚡️

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
model.push_to_hub('medical-literature-small')
tokenizer.push_to_hub('medical-literature-small')

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/gokcenazakyol/medical-literature-small/commit/3fac2539b85b7abec3071b3bd2b5bb9e04d35150', commit_message='Upload tokenizer', commit_description='', oid='3fac2539b85b7abec3071b3bd2b5bb9e04d35150', pr_url=None, pr_revision=None, pr_num=None)